## Indeed webscraping

In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd

Delete the job text and location in the url and replace it with curly braces.
Create a string template that you'll use to create whatever job position or location that you want.

In [2]:
template = 'https://nl.indeed.com/jobs?q={}&l={}'

Create a function called get_url, with the arguments of position and location. Then move the template variable inside this function.

In [3]:
def get_url(position, location):
    """Generate a url from position and location"""
    template = 'https://nl.indeed.com/jobs?q={}&l={}'
    url = template.format(position, location)
    return url

In [4]:
url = get_url('data analist', 'Nederland')

### Extract raw html

In [5]:
response = requests.get(url)

Extract the raw html from the response object, then parse this with Beautifulsoup.

In [6]:
soup = BeautifulSoup(response.text, 'html.parser')

Find the html tag that incloses the entire record: 'jobsearch-SerpJobCard'

In [7]:
cards = soup.find_all('div', 'jobsearch-SerpJobCard')

Check the number of job postings on the page.

In [8]:
len(cards)

15

### Prototype the model with a single record

In [9]:
card = cards[0]

In [10]:
atag = card.h2.a

In [11]:
job_title = atag.get('title')
job_title

'Data Analist'

In [12]:
job_url = 'https://nl.indeed.com' + atag.get('href')
job_url

'https://nl.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0BUr1ih89J8xlxnhfvrvaSPyfeHEUaTmnzB_AydzFDKnIiZ_IebOX6YnjJSMiF7dRLn9puMMVtEjWmhzpOVn_3hDp8cIFVXzTAYKweJColTCEpTwGwYCi6oDI3fy3ehYQTqneslOaS-XtBKDfnTdVm9teqlM0UWNpg0kzVTij43bHAQsqWOyNjZ9I4Hfz6ZH1d9Qvco994O8Qcr2Sj1x1EpK8HGgGmOa9G4PSlRuqmopAdgr3zKz8uUxoucgi3zhngecKCxaqE-hzSalAkn80lE5rIJLQTSF5yA9fMVraexT9vjZraYfJ_UmaDP_lNatuplUNC3oMcKMrGx2jJGXhobZW0gtzhKb0PkqNC_7Pzg_R3Co5wzFYNMgU5chSxNjVF3MGT3qdMRbcUePwXDGBVTzxM9MiN6O2DydCdSljasyQIWD39AZjkoR3turiNXb2wOBJWqoUDLt-04Ohjxtvd4dc7DKEaiXeKILMS90lhaSX3FlnwYbTE7pgHe7EhE-nGi_4nI_FuJwWbNp6jxrkQo64mTfF8T-ci4Wmjkm98BWr2xObt6eh4CqAPu14rP0ZMTdDqLVBtysYZKvwQDZ62Kllx1nNrOhXFYIHqZbWH2qQ3NK_wRF247f902Nejxb60=&p=0&fvj=0&vjs=3'

In [13]:
company = card.find('span', 'company').text.strip()
company

'Aon'

In [14]:
job_location = card.find('div', 'recJobLoc').get('data-rc-loc')
job_location

'Eindhoven'

In [15]:
job_summary = card.find('div', 'summary').text.strip()
job_summary

'Verzamelen van data uit zowel interne als externe bronnen (UWV, CPB, CBS).\nUitvoeren van analyses aan de hand van data, referentie-onderzoeken, gesprekken en…'

In [16]:
post_date = card.find('span', 'date').text
post_date

'Meer dan 30 dagen geleden'

Also grab the current date, so that you have something to compare to the relative data you get from the job posting.

In [17]:
today = datetime.today().strftime('%Y-%m-%d')
today

'2021-03-13'

The salary range is not available for all job postings.

In [18]:
try:
    job_salary = card.find('span', 'salaryText').text.strip()
except AttributeError:
    job_salary = ''
    
job_salary

'€2.600 - €3.500 per maand'

In [ ]:
card = cards[0]
id = card.find('div', 'recJobLoc').get('id').split('_')[1]
id

### Generalize the model with a function

Create a function called get_record, which will accept a single argument card.
Include the code created into this function.

In [ ]:
def get_record(card):
    """Extract job data from a single record"""
    id = card.find('div', 'recJobLoc').get('id').split('_')[1]
    atag = card.h2.a
    job_title = atag.get('title')
    job_url = 'https://nl.indeed.com' + atag.get('href')
    company = card.find('span', 'company').text.strip()
    job_location = card.find('div', 'recJobLoc').get('data-rc-loc')
    job_summary = card.find('div', 'summary').text.strip()
    post_date = card.find('span', 'date').text
    today = datetime.today().strftime('%Y-%m-%d')
    try:
        job_salary = card.find('span', 'salaryText').text.strip()
    except AttributeError:
        job_salary = ''
        
    record = {'id':id, 'title':title, 'company':company, 'location':location,
              'postingdate':postingdate, 'today':today, 'summary':summary,
              'salary':salary, 'url':url}
    
    return record

Create a list called records. Then, iterate through each card, extracting the record from the card data and then appending that extracted data to the records list.

In [ ]:
records = []

for card in cards:
    record = get_record(card)
    records.append(record)

Check out a few items in the records list.

In [ ]:
records[0]

### Getting the next page

- Go to the page and click right to inspect the chevron.
- If the program can't find the tag it will return an AttributeError
- Create a while loop that continues to run until this url returns an AttributeError, at which point break out of the loop
- While being in the loop, execute all of the code you've written up to this point

In [ ]:
while True:
    try:
        url = 'https://nl.indeed.com' + soup.find('a',{'aria-label': 'Volgende'}).get('href')
    except AttributeError:
        break
    
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    cards = soup.find_all('div', 'jobsearch-SerpJobCard')
    
    for card in cards:
        record = get_record(card)
        records.append(record)

In [ ]:
len(records)

In [ ]:
# create dataframe
listings_df = pd.DataFrame(records, columns=['Title', 'Company', 'Location', 'Date', 'Scrape Date', 'Summary', 'Salary', 'Url'])

# check if works
listings_df.head()

In [ ]:
# export dataframe to csv
listings_df.to_csv('listings2.csv')

# TO DO:

- Add ratings?

- Check for job skills (e.g. create list with different programming languages and count how often they appear)

- Job urls not working?